In [1]:
import geopandas as gpd
import pandas as pd
# import pydeck as pdk
from glob import glob
import numpy as np
from scipy import sparse

In [2]:
# !pip install pydeck
# !pip install geopandas

In [3]:
# tracts = gpd.read_file('./data/census_tracts.zip')
# tract_centroids = tracts
# tract_centroids['geometry'] = tracts.centroid
# tract_centroids.to_csv('./data/tract_centroids.csv')
tract_centroids = pd.read_csv('./data/tract_centroids.csv')

In [4]:
centroid_dict = {}
for i in range(0, len(tract_centroids)):
    centroid_dict[tract_centroids.iloc[i].GEOID] = {
        "lat": round(tract_centroids.iloc[i].INTPTLAT, 4),
        "lon": round(tract_centroids.iloc[i].INTPTLON, 4),
    }

In [5]:
tract_ids = [value[0] for value in pd.read_csv('./data/chicago/TractID_List.csv', header=None).values]


In [7]:
timeseries_files = glob('./data/chicago/*.csv')
dates = []

for file in timeseries_files:
    if 'TractID_List' in file:
        continue
    else:
        dates.append(file.split('/')[-1].split('.')[0])

In [ ]:
sparse = []

for i in range(0, len(timeseries_files)):
    file = timeseries_files[0]
    date = file.split('/')[-1].split('.')[0]
    df = pd.read_csv(file, header=None)
    vals = df.values
    for i in range(0, len(vals)):
        for j in range(0, len(vals[i])):
            o = tract_ids[i]
            d = tract_ids[j]
            sparse.append({
                "o_lat": centroid_dict[o]["lat"],
                "o_lon": centroid_dict[o]["lon"],
                "d_lat": centroid_dict[d]["lat"],
                "d_lon": centroid_dict[d]["lon"],
                "count": vals[i][j],
                "date": date
            })

sparse_df = pd.DataFrame(sparse)
sparse_df[sparse_df['count'] > 0].to_csv('./data/sparse_chicago.csv', index=False)

In [ ]:
sparse_df.head()